# Imports

In [1]:
#imports
import pandas as pd
import re
from datetime import date

# Read and Clean Data


*   Demographics - survey_data
*   Ridership - ridership_2019data / 
ridership_2020data
*   Location Data - stop_loc






In [2]:
# reads survey data and convert from excel file to csv
# first sheet by default
survey_data = pd.read_excel(r'survey.xlsx')
# read_file.to_csv(r'survey.csv', index = None, header=True)

#reads ridership data
ridership_2019data = pd.read_csv(r'ridership19.csv')
#change service date to date type
ridership_2019data['service_date'] = pd.to_datetime(ridership_2019data['service_date'], format="%Y/%m/%d")

ridership_2020data = pd.read_csv(r'ridership20.csv')
#change service date to date type
ridership_2020data['service_date'] = pd.to_datetime(ridership_2020data['service_date'], format="%Y/%m/%d")

#reads stop data
stop_loc = pd.read_csv(r'stop_loc.csv')

In [3]:
#Filter out rows if Mode is "Commuter Rail"
survey_data = survey_data.loc[survey_data['Mode'] != "Commuter Rail"]

#Filter out stations with long term constructions
constructionstops = ['Wollaston', 'Science Park', 'Lechmere']
ridership_2020data = ridership_2020data.loc[~ridership_2020data['station_name'].isin(constructionstops)]
ridership_2019data = ridership_2019data.loc[~ridership_2019data['station_name'].isin(constructionstops)]

# Joining 2019 and 2020 Ridership Data
*   By Year       - joineddf
*   By Month      - joinedmonthdf
*   By WeekofYear - joinedwoydf

In [4]:
def addweekofyear_df(df):
  df['Day'] = df["service_date"].dt.day
  df['Month'] = df["service_date"].dt.month
  df['Year'] = df["service_date"].dt.year
  df['WeekOfYear'] = df['service_date'].dt.isocalendar().week
  # Finds the year associated with the Week of Year
  df['WeekYear'] = df['service_date'].dt.isocalendar().year

def aggregate(df, listofaggbycolumn, aggfunction, aggcol):
  """ performs an aggregate function by the given columns list in the data
  frame on the specified column """
  # Removing as index to become dataframe
  df2 = df.groupby(listofaggbycolumn).agg(aggfunction)[aggcol]
  return df2.reset_index()
  
def addpercentchange_df(df):
  ''' takes df with columns ['gated_entries2019', 'gated_entries2020']  adds columns with percent change'''
  # as a float/decimal
  df['PercentChange'] = (df['gated_entries2020']-df['gated_entries2019'])/df['gated_entries2019']

In [5]:
# Add date related columns
addweekofyear_df(ridership_2019data)
addweekofyear_df(ridership_2020data)

In [6]:
# Aggregate by Year, Station, Line
agg_2019  = aggregate(ridership_2020data, ["route_or_line", "station_name", "stop_id"], sum, "gated_entries")
agg_2020  = aggregate(ridership_2019data, ["route_or_line", "station_name", "stop_id"], sum, "gated_entries")
# Join both years to compare 
joineddf = agg_2019.merge(agg_2020, how='outer', on=["route_or_line", "station_name", "stop_id"], suffixes=['2019','2020'])
# Add Percent Change
addpercentchange_df(joineddf)

In [7]:
# Aggregate by Month, Station, Line
aggmonth_2019  = aggregate(ridership_2020data, ["Month", "route_or_line", "station_name", "stop_id"], sum, "gated_entries")
aggmonth_2020  = aggregate(ridership_2019data, ["Month", "route_or_line", "station_name", "stop_id"], sum, "gated_entries")
# Join both years to compare 
joinedmonthdf = aggmonth_2019.merge(aggmonth_2020, how='outer', on=["Month", "route_or_line", "station_name", "stop_id"], suffixes=['2019','2020']).dropna()
# Add Percent Change
addpercentchange_df(joinedmonthdf)

In [10]:
#Aggregate by Week of Year, Station, Line
# Merge 2019 and 2020 (to adjust for week of year overlaps)
week_merge = pd.concat([ridership_2019data, ridership_2020data])
# Aggregate by Week of Year and Corresponding Year
aggwoy_merged = aggregate(week_merge
                          , ["WeekOfYear", 'WeekYear', "route_or_line"
                          , "station_name", "stop_id"]
                          , sum , "gated_entries")
# Filter by Year
woy_2019 = aggwoy_merged.loc[aggwoy_merged['WeekYear']== 2019]
woy_2020 = aggwoy_merged.loc[aggwoy_merged['WeekYear'] == 2020]
# Join both years to compare (NaNs due to longer 2020 year)
joinedwoydf = woy_2019.merge(woy_2020, how='outer', on=["WeekOfYear", "route_or_line", "station_name", "stop_id"], suffixes=['2019','2020']).reset_index().dropna()
# Add Percent Change
addpercentchange_df(joinedwoydf)

# get rid WeekYear 
joinedmonthdf

,Month,route_or_line,station_name,stop_id,gated_entries2019,gated_entries2020,PercentChange
0,1,Blue Line,Airport,place-aport,172599.45,177166.0,0.026458
1,1,Blue Line,Aquarium,place-aqucl,172062.48,105869.0,-0.384706
2,1,Blue Line,Beachmont,place-bmmnl,164819.86,74198.0,-0.549824
3,1,Blue Line,Bowdoin,place-bomnl,169377.81,49927.0,-0.705233
4,1,Blue Line,Government Center,place-gover,167040.13,153597.6,-0.080475
...,...,...,...,...,...,...,...
811,12,Red Line,Shawmut,place-smmnl,39395.47,44679.0,0.134115
812,12,Red Line,South Station,place-sstat,42140.21,338223.2,7.026139
813,12,Silver Line,Courthouse,place-crtst,41249.18,74181.0,0.798363
814,12,Silver Line,South Station,place-sstat,41259.86,144952.8,2.513168
